In [1]:
import pandas as pd
import os
import requests
import numpy as np
import json

In [44]:
import tweepy

# Read in the config file with your personal keys, secrets, and tokens
with open('config_file.txt') as file:
    lines = file.readlines()
    consumer_key = lines[0].strip('\n')
    consumer_secret = lines[1].strip('\n')
    access_token = lines[2].strip('\n')
    access_secret = lines[3].strip('\n')
    
# Set up the API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [4]:
# Read in twitter archive
tweets = pd.read_csv('twitter-archive-enhanced.csv')

In [6]:
# Grab the image predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
images = requests.get(url)

In [24]:
# Create tsv of images
with open('images.csv', 'x') as file:
    file.write(images.text)

In [27]:
images_df = pd.read_csv('images.csv', sep = '\t')

In [55]:
file_name = 'tweet_json.txt'
errors = {}
with open(file_name, 'x') as outfile:
    for tweet in tweets['tweet_id']:
        try:
            tweet_json = api.get_status(tweet)._json
            json.dump(tweet_json, outfile)
            outfile.write('\n')
        except Exception as e:
            print(tweet)
            errors[str(tweet)] = 'Tweet deleted'


888202515573088257
873697596434513921
869988702071779329
866816280283807744
861769973181624320
842892208864923648
827228250799742977
802247111496568832
775096608509886464
Rate limit reached. Sleeping for: 152
Rate limit reached. Sleeping for: 687


In [88]:
tweet_list = []

with open(file_name) as infile:
    for line in infile:
        data = json.loads(line)
        tweet_id = data['id_str']
        favorites = data['favorite_count']
        retweets = data['retweet_count']
        tweet_list.append({'tweet_id': tweet_id,
                           'favorite_count': favorites,
                           'retweet_count': retweets})
        
json_df = pd.DataFrame(tweet_list, columns = ['tweet_id', 'favorite_count', 'retweet_count'])

In [89]:
json_df.head()

,tweet_id,favorite_count,retweet_count
0,892420643555336193,39148,8710
1,892177421306343426,33532,6391
2,891815181378084864,25273,4245
3,891689557279858688,42514,8806
4,891327558926688256,40674,9590
